In [23]:
#Imports

import pandas as pd 
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier



In [39]:
def convert_sex(val):
    if val == 'male': return 1
    else: return 0
    
def read_data(file):
    return pd.read_csv(file, sep=',', converters = {'Sex': convert_sex})

#Read in CSVs
train_data = read_data('train.csv')
test_data = read_data('test.csv')

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#Print info on dataframes extracted from csv files for train/test
train_data.info()
print("----------------------------")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(6), object(4)
memory usage: 83.6+ KB
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare  

In [3]:
#Columns that don't appear to have any relevance towards prediction
del_columns = ['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked']

#Remove unwanted columns
train_data = train_data.drop(del_columns, axis = 1)
test_data = test_data.drop(del_columns, axis = 1)

#Remove NaN rows
train_data = train_data.dropna()
test_data = test_data.dropna()

In [8]:
#Split datasets into training data(X),  and classifier(y)
train_y = train_data['Survived']
train_X = train_data.drop("Survived", axis=1)
test_X = test_data


In [36]:
#Naive Bayes
gnb = GaussianNB()
gnb.fit(train_X, train_y)

train_y_pred = gnb.predict(train_X)
test_y_pred = gnb.predict(test_X)

#Compare acccuracy of prediction
#print(metrics.accuracy_score(train_y, train_y_pred))
print(gnb.score(train_X, train_y))

0.788515406162


In [35]:
#Logistic Regression
lr = LogisticRegression()
lr.fit(train_X, train_y)

train_y_pred = lr.predict(train_X)
test_y_pred = lr.predict(test_X)

#Compare acccuracy of prediction
#print(metrics.accuracy_score(train_y, train_y_pred))
print(lr.score(train_X, train_y))

0.789915966387


In [38]:
#RandomForest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_X, train_y)

train_y_pred = rf.predict(train_X)
test_y_pred = rf.predict(test_X)

#Compare acccuracy of prediction
#print(metrics.accuracy_score(train_y, train_y_pred))
print(rf.score(train_X, train_y))

0.932773109244


In [33]:
#RandomForest
svc = SVC()
svc.fit(train_X, train_y)

train_y_pred = svc.predict(train_X)
test_y_pred = svc.predict(test_X)

#Compare acccuracy of prediction
#print(metrics.accuracy_score(train_y, train_y_pred))
print(svc.score(train_X, train_y))

0.872549019608


In [32]:
#K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_X, train_y)

train_y_pred = knn.predict(train_X)
test_y_pred = knn.predict(test_X)

#Compare acccuracy of prediction
#print(metrics.accuracy_score(train_y, train_y_pred))
print(knn.score(train_X, train_y))

0.862745098039
